# Getting "context chunks" and topic modeling those chunks

This notebook is to find the character start and end indices for a given quotation, then generate a csv file with all items quoting within that range.

## Import

In [1]:
from text_matcher.matcher import Text, Matcher
import json
#import nltk
#nltk.download('stopwords')
import pandas as pd
from IPython.display import clear_output
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 6]
#pd.set_option('display.max_colwidth', None)

## Define filepath

In [2]:
# ACTION: copy path to results JSONL file here (filename should end "_results_[hyperparameters].jsonl")

startData = "/Users/milan/Library/CloudStorage/GoogleDrive-mtt2126@columbia.edu/My Drive/iAnnotate/MIT/Quotable Content/Data/Barthes/1977_DeathAuthorHeath/Results/Barthes_1977_DeathAuthorHeath_results_t2-c3-n2-m3-nostops.jsonl"

In [3]:
# Infer naming variables from path

textTitle = startData.rsplit("_", 4)[-3]
publicationYear = startData.rsplit("_", 4)[-4]
authorSurname = startData.rsplit("_", 4)[-5]
authorSurname = authorSurname.rsplit("/", 1)[-1]
hyperparSuffix = startData.rsplit("_", 4)[-1]
hyperparSuffix = f"_{hyperparSuffix[:-6]}"
dataDir = startData.rsplit("/", 4)[0]

print(f"Author surname: {authorSurname}\nPublication year: {publicationYear}\nText title: {textTitle}\nHyperparameters suffix: {hyperparSuffix}\nData directory:{dataDir}")

projectName = f"{authorSurname}_{publicationYear}_{textTitle}"
sourceDir = f"{dataDir}/{authorSurname}/{publicationYear}_{textTitle}/SourceText"
corpusDir = f"{dataDir}/{authorSurname}/{publicationYear}_{textTitle}/TargetCorpus"
resultsDir = f"{dataDir}/{authorSurname}/{publicationYear}_{textTitle}/Results"

Author surname: Barthes
Publication year: 1977
Text title: DeathAuthorHeath
Hyperparameters suffix: _t2-c3-n2-m3-nostops
Data directory:/Users/milan/Library/CloudStorage/GoogleDrive-mtt2126@columbia.edu/My Drive/iAnnotate/MIT/Quotable Content/Data


In [4]:
# Load .txt file 
inputText = f"{sourceDir}/{projectName}_plaintext.txt"
with open(inputText) as f: 
    txt = f.read()

processedText = Text(txt, f'{textTitle}')

In [5]:
# Load in the JSON file with our JSTOR articles and data from TextMatcher
# (Note: must have the file 'default.json' in the same directory as this notebook)
# Load results as pandas dataframe

df = pd.read_json(startData, lines=True)
df

,creator,datePublished,Year,Decade,docSubType,docType,doi,id,identifier,isPartOf,...,title,url,volumeNumber,wordCount,numMatches,Locations in A,Locations in B,abstract,placeOfPublication,subTitle
0,"[ROSEMARIE MORGAN, T. R. Wright]",1990-06-02,1990,1990,book-review,article,10.2307/45274001,http://www.jstor.org/stable/45274001,"[{'name': 'doi', 'value': '10.2307/45274001'},...",The Thomas Hardy Journal,...,HARDY AND THE EROTIC,http://www.jstor.org/stable/45274001,6,1741,0,[],[],None,None,None
1,[Karen L. Thornber],2009-08-01,2009,2000,research-article,article,10.2307/20619794,http://www.jstor.org/stable/20619794,"[{'name': 'doi', 'value': '10.2307/20619794'},...",The Journal of Asian Studies,...,Early Twentieth-Century Intra—East Asian Liter...,http://www.jstor.org/stable/20619794,68,12625,0,[],[],None,None,None
2,[MARTIN THOMAS],2009-12-01,2009,2000,research-article,article,10.2307/25643867,http://www.jstor.org/stable/25643867,"[{'name': 'doi', 'value': '10.2307/25643867'},...",The Historical Journal,...,FRENCH EMPIRE ELITES AND THE POLITICS OF ECONO...,http://www.jstor.org/stable/25643867,52,14057,0,[],[],This article considers the changing ways in wh...,None,None
3,"[Michael Allen, Paul Muldoon]",1995-12-01,1995,1990,book-review,article,10.2307/29735794,http://www.jstor.org/stable/29735794,"[{'name': 'doi', 'value': '10.2307/29735794'},...",The Irish Review (1986-),...,An Extended Conceit,http://www.jstor.org/stable/29735794,None,2028,0,[],[],None,None,None
4,[SAAM TRIVEDI],2015-10-01,2015,2010,research-article,article,10.2307/24544207,http://www.jstor.org/stable/24544207,"[{'name': 'doi', 'value': '10.2307/24544207'},...",College Literature,...,"SURPLUS, AUTHORIAL INTENTIONS, AND HYPOTHETICA...",http://www.jstor.org/stable/24544207,42,11383,0,[],[],I begin in the first section of this essay by ...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20251,[Daniel Chávez Landeros],2023-01-01,2023,2020,book,book,10.2307/jj.10984428,http://www.jstor.org/stable/jj.10984428,"[{'name': 'doi', 'value': '10.2307/jj.10984428...",None,...,Fábula del Poder,http://www.jstor.org/stable/jj.10984428,None,131276,0,[],[],None,"West Lafayette, Indiana","Corporalidad, biopolítica y violencia en la na..."
20252,[MICHAEL THORPE],1995-10-01,1995,1990,research-article,article,10.2307/45274820,http://www.jstor.org/stable/45274820,"[{'name': 'doi', 'value': '10.2307/45274820'},...",The Thomas Hardy Journal,...,SUE THE OBSCURE: Hardy's Female Readers,http://www.jstor.org/stable/45274820,11,5308,0,[],[],None,None,None
20253,"[Stephen Steele, André Breton, Jean-Pierre Sak...",2004-04-01,2004,2000,book-review,article,10.2307/40838362,http://www.jstor.org/stable/40838362,"[{'name': 'doi', 'value': '10.2307/40838362'},...",Dalhousie French Studies,...,"42, rue Fontaine (DVD-ROM)",http://www.jstor.org/stable/40838362,66,1214,0,[],[],None,None,None
20254,[Gaynor Macdonald],2003-06-01,2003,2000,research-article,article,10.2307/40331904,http://www.jstor.org/stable/40331904,"[{'name': 'doi', 'value': '10.2307/40331904'},...",Oceania,...,Photos in Wiradjuri Biscuit Tins: Negotiating ...,http://www.jstor.org/stable/40331904,73,11213,0,[],[],"Tins, cardboard boxes and albums hold one of t...",None,None


In [6]:
# Drop items with no matches from dataframe

df = df[df['numMatches']>=1]
df

,creator,datePublished,Year,Decade,docSubType,docType,doi,id,identifier,isPartOf,...,title,url,volumeNumber,wordCount,numMatches,Locations in A,Locations in B,abstract,placeOfPublication,subTitle
43,[Frank Burke],1989-10-01,1989,1980,research-article,article,10.2307/1212711,http://www.jstor.org/stable/1212711,"[{'name': 'doi', 'value': '10.2307/1212711'}, ...",Film Quarterly,...,Fellini: Changing the Subject,http://www.jstor.org/stable/1212711,43,9433,1,"[[6765, 6841]]","[[500, 572]]",None,None,None
50,[Lars-Olof Åhlberg],1999-04-01,1999,1990,research-article,article,10.2307/3333732,http://www.jstor.org/stable/3333732,"[{'name': 'doi', 'value': '10.2307/3333732'}, ...",Journal of Aesthetic Education,...,Understanding and Appreciating Art: The Releva...,http://www.jstor.org/stable/3333732,33,6263,2,"[[3009, 3138], [10160, 10357]]","[[3946, 4075], [4380, 4623]]",None,None,None
79,[Leila Rahimi Bahmany],2015-01-01,2015,2010,book,book,10.2307/j.ctv312d0np,http://www.jstor.org/stable/j.ctv312d0np,"[{'name': 'doi', 'value': '10.2307/j.ctv312d0n...",None,...,Mirrors of Entrapment and Emancipation,http://www.jstor.org/stable/j.ctv312d0np,None,137870,1,"[[727, 883]]","[[815979, 816134]]",None,None,Forugh Farrokhzad and Sylvia Plath
93,[SEÁN TRAVERS],2018-01-01,2018,2010,research-article,article,10.2307/26489194,http://www.jstor.org/stable/26489194,"[{'name': 'doi', 'value': '10.2307/26489194'},...",IJAS Online,...,EMPTY CONSTRUCTS,http://www.jstor.org/stable/26489194,None,6918,1,"[[9055, 9095]]","[[4737, 4777]]",None,None,None
104,[Kinereth Meyer],1989-07-01,1989,1980,research-article,article,10.2307/41153396,http://www.jstor.org/stable/41153396,"[{'name': 'doi', 'value': '10.2307/41153396'},...",Comparative Drama,...,'It Is Written': Tom Stoppard and the Drama of...,http://www.jstor.org/stable/41153396,23,7806,1,"[[642, 883]]","[[13252, 13493]]",None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20138,"[Fred Misurella, Alvin J. Seltzer]",1975-10-01,1975,1970,book-review,article,10.2307/1345027,http://www.jstor.org/stable/1345027,"[{'name': 'doi', 'value': '10.2307/1345027'}, ...",NOVEL: A Forum on Fiction,...,Contemporary Chaos,http://www.jstor.org/stable/1345027,9,1274,1,"[[13446, 13506]]","[[5122, 5185]]",None,None,None
20172,[Margarita Tupitsyn],1993-12-01,1993,1990,research-article,article,10.2307/777620,http://www.jstor.org/stable/777620,"[{'name': 'doi', 'value': '10.2307/777620'}, {...",Art Journal,...,Collaborating on the Paradigm of the Future,http://www.jstor.org/stable/777620,52,5235,2,"[[8539, 8615], [8646, 8840]]","[[19749, 19824], [19856, 20049]]",None,None,None
20187,[Judith May Fathallah],2017-01-01,2017,2010,chapter,chapter,10.2307/j.ctt1v2xsp4.9,http://www.jstor.org/stable/j.ctt1v2xsp4.9,"[{'name': 'doi', 'value': '10.2307/j.ctt1v2xsp...",Fanfiction and the Author,...,‘I’m a God’:,http://www.jstor.org/stable/j.ctt1v2xsp4.9,None,17777,1,"[[657, 709]]","[[45016, 45068]]","<p>As we have observed, cult television makes ...",Amsterdam,The Author and the Writing Fan in Supernatural
20210,"[MARK L. SAMPLE, Matthew K. Gold]",2012-01-01,2012,2010,chapter,chapter,10.5749/j.ctttv8hq.14,http://www.jstor.org/stable/10.5749/j.ctttv8hq.14,"[{'name': 'doi', 'value': '10.5749/j.ctttv8hq....",Debates in the Digital Humanities,...,Unseen and Unremarked On:,http://www.jstor.org/stable/10.5749/j.ctttv8hq.14,None,6320,1,"[[7151, 7203]]","[[18312, 18370]]",<p>Like Don DeLillo’s professor of latent hist...,Minneapolis,Don DeLillo and the Failure of the Digital Hum...


## Let's find the start and end characters for a given quotation

In [7]:
processedText.text

'In his story Sarrasine Balzac, describing a castrato disguised as a woman, writes the following sentence: \'This was woman herself, with her sudden fears, her irrational whims, her instinctive worries, her impetuous boldness, her fussings, and her delicious sensibility.\' Who is speaking thus? Is it the hero of the story bent on remaining ignorant of the castrato hidden beneath the woman? Is it Balzac the individual, furnished by his personal experience with a philosophy of Woman? Is it Balzac the author professing \'literary\' ideas on femininity? Is it universal wisdom? Romantic psychology? We shall never know, for the good reason that writing is the destruction of every voice, of every point of origin. Writing is that neutral, composite, oblique space where our subject slips away, the negative where all identity is lost, starting with the very identity of the body writing.\n\nNo doubt it has always been that way. As soon as a fact is narrated no longer with a view to acting directl

In [8]:
# ACTION: replace string between quotation marks with desired quotation
quotation = 'We know now that a text is not a line of words releasing a single \'theological\' meaning (the \'message\' of the Author-God) but a multi-dimensional space in which a variety of writings, none of them original, blend and clash'

# ACTION: give this passage a shorthand name
passageShorthand = "multidimensionalSpace"


In [9]:
index = processedText.text.index(quotation)

passageStart = index
passageEnd = index + len(quotation)
quoteIndex = [passageStart, passageEnd]

print('Starting index:', passageStart) 
print('Ending index:', passageEnd)
print(f'Character indexes for match: {quoteIndex}')
print("\n Corresponding text:")
processedText.text[index:index + len(quotation)]

Starting index: 8536
Ending index: 8758
Character indexes for match: [8536, 8758]

 Corresponding text:


"We know now that a text is not a line of words releasing a single 'theological' meaning (the 'message' of the Author-God) but a multi-dimensional space in which a variety of writings, none of them original, blend and clash"

In [10]:
# Let's look just at just our character indexes for quotations
df = df[['id', 'Year', 'Locations in A', 'Locations in B']]
df

,id,Year,Locations in A,Locations in B
43,http://www.jstor.org/stable/1212711,1989,"[[6765, 6841]]","[[500, 572]]"
50,http://www.jstor.org/stable/3333732,1999,"[[3009, 3138], [10160, 10357]]","[[3946, 4075], [4380, 4623]]"
79,http://www.jstor.org/stable/j.ctv312d0np,2015,"[[727, 883]]","[[815979, 816134]]"
93,http://www.jstor.org/stable/26489194,2018,"[[9055, 9095]]","[[4737, 4777]]"
104,http://www.jstor.org/stable/41153396,1989,"[[642, 883]]","[[13252, 13493]]"
...,...,...,...,...
20138,http://www.jstor.org/stable/1345027,1975,"[[13446, 13506]]","[[5122, 5185]]"
20172,http://www.jstor.org/stable/777620,1993,"[[8539, 8615], [8646, 8840]]","[[19749, 19824], [19856, 20049]]"
20187,http://www.jstor.org/stable/j.ctt1v2xsp4.9,2017,"[[657, 709]]","[[45016, 45068]]"
20210,http://www.jstor.org/stable/10.5749/j.ctttv8hq.14,2012,"[[7151, 7203]]","[[18312, 18370]]"


In [11]:
# Loop over DF to get ID and each individidual match in Locations in B and each in B
df = df.explode(['Locations in A', 'Locations in B'])


### Let's get all the ids, Locations in B and Locations in A for just our specified character index range (eg, squirrel's heartbeat

In [12]:
# Define a function that can compare two intervals in the form of listed tuples, eg [x0, x1], [y0,y1]
def overlaps(range1, range2):
    try:
        if len(range1) == 2 and len(range2) == 2:
            return max(range1[0],range2[0]) < min(range1[1],range2[1])
        else:
            return False
    except TypeError:
        return False

In [13]:
quotation_locations_in_A = []
quotation_locations_in_B = []
ids = []
years = []


# Let's extract just the Locations in A that overlap with the squirrel's hearbeat quotation
for item in range(len(df)):
    loc_in_a_index = df['Locations in A'].iloc[item]
    if overlaps(quoteIndex, loc_in_a_index) == True:
        quotation_locations_in_A.append(df['Locations in A'].iloc[item])
        quotation_locations_in_B.append(df['Locations in B'].iloc[item])
        ids.append(df['id'].iloc[item])
        years.append(df['Year'].iloc[item])

In [14]:
zipped = list(zip(ids, years, quotation_locations_in_A, quotation_locations_in_B))
just_matches_df = pd.DataFrame(zipped, columns=['id', 'Year', 'Locations in A', 'Locations in B'])
just_matches_df

,id,Year,Locations in A,Locations in B
0,http://www.jstor.org/stable/42946282,1995,"[8664, 8758]","[33328, 33422]"
1,http://www.jstor.org/stable/10.3998/mpub.12697...,2023,"[8569, 8615]","[38358, 38403]"
2,http://www.jstor.org/stable/3045790,1991,"[8539, 8656]","[51124, 51241]"
3,http://www.jstor.org/stable/j.ctt1zkjz0m,2017,"[8743, 8794]","[740562, 740613]"
4,http://www.jstor.org/stable/29790443,1997,"[8539, 8615]","[46727, 46802]"
...,...,...,...,...
134,http://www.jstor.org/stable/44377024,1988,"[8646, 8687]","[4612, 4658]"
135,http://www.jstor.org/stable/10.5250/womgeryear...,2012,"[8664, 8758]","[10775, 10869]"
136,http://www.jstor.org/stable/26813050,1992,"[8539, 8758]","[55926, 56145]"
137,http://www.jstor.org/stable/777620,1993,"[8539, 8615]","[19749, 19824]"


In [15]:
just_matches_df.to_csv(f"{resultsDir}/{textTitle}_{passageShorthand}.csv")

# Alternate for VS


In [ ]:
quotation_locations_in_A = []
quotation_locations_in_B = []
ids = []

for item in range(len(df_just_quotes)):
    loc_in_a_index = df_just_quotes['Locations in A'].iloc[item]
    quotation_locations_in_A.append(df_just_quotes['Locations in A'].iloc[item])
    quotation_locations_in_B.append(df_just_quotes['Locations in B'].iloc[item])
    ids.append(df_just_quotes['id'].iloc[item])

### Let's use the start and end characters to extract a context chunk left & right of our quotations

Set the `num_characters_before_quote` and `num_characters_after_quote`

In [ ]:
# Use article_id to get the index of the article in our DataFrame
df_with_fulltexts = pd.read_json('default.json')
df_with_fulltexts['year'] = pd.DatetimeIndex(df_with_fulltexts['datePublished']).year
df_with_fulltexts['Decade'] = df_with_fulltexts['year'] - (df_with_fulltexts['year'] % 10)

In [ ]:
# Change these to desired chunk
num_characters_before_quote = 200
num_characters_after_quote = 750
decades_to_check = [2010, 2000, 1990, 1980, 1970, 1960, 1950]
list_of_VS_journals = ['Victorian Studies', 'George Eliot - George Henry Lewes Studies', 'Nineteenth-Century Fiction', 'Nineteenth-Century Literature', 'Dickens Studies Annual', 'Victorian Literature and Culture', 'Victorian Review', 'The George Eliot, George Henry Lewes Newsletter', 'Victorian Periodicals Review', 'Dickens Quarterly', 'Victorian Poetry', 'The Thomas Hardy Journal', 'The Gaskell Society Journal', 'The Gaskell Journal', 'Newsletter of the Victorian Studies Association of Western Canada', 'Dickens Studies Newsletter', 'Browning Institute Studies', 'Victorian Periodicals Newsletter', 'Carlyle Studies Annual', 'Conradiana', 'Tennyson Research Bulletin', 'The Conradian', 'The Hardy Society Journal', 'The Hardy Review', 'Studies in Browning and His Circle', 'Nineteenth-Century French Studies', 'The Wilkie Collins Journal', 'Carlyle Newsletter', 'The Wildean', 'Dickens Studies', 'Carlyle Annual', '19th-Century Music', 'The Trollopian', 'Conrad Studies']

# Create an empty list that we will populate with the contexts for quotations
context_chunks_for_quotations = []
context_chunks_ids = []
context_chunks_decade = []


# Loop over each of the start and end locations to produce a x-character chunk of context
for item in range(len(just_matches_df)):
    article_URL = just_matches_df['id'].loc[item]
    startandEndLocations = just_matches_df['Locations in B'].loc[item]
    print(article_URL)
    print(startandEndLocations[0])
    print(startandEndLocations[1])
    article_index = df_with_fulltexts[df_with_fulltexts['id'] == article_URL].index[0]
    print(article_index)
    article_title = df_with_fulltexts['title'].loc[article_index]
    print(article_title)
    article_decade = df_with_fulltexts['Decade'].loc[article_index]
    print(article_decade)
    article_journal = df_with_fulltexts['isPartOf'].loc[article_index]
    article_text = df_with_fulltexts['fullText'].loc[article_index]
    cleaned_article_text = Text(article_text, article_title)
    if article_journal in list_of_VS_journals:
    #if article_decade in decades_to_check:
        context_chunks_for_quotations.append((cleaned_article_text.text[startandEndLocations[0]-num_characters_before_quote:startandEndLocations[0]]) + "[...]" + (cleaned_article_text.text[startandEndLocations[1]:startandEndLocations[1]+num_characters_after_quote]))
        context_chunks_ids.append(df_with_fulltexts['id'].loc[article_index])
        context_chunks_decade.append(df_with_fulltexts['Decade'].loc[article_index])

### Print the contexts around quotations:

In [ ]:
context_chunks_for_quotations

In [ ]:
context_chunks_ids

### Print the context around the fifth quotation

In [ ]:
print(context_chunks_for_quotations[3])

## To write output to a single text file

In [ ]:
name_of_quote = f"{name_of_passage}-{num_characters_before_quote}-char-before-and-{num_characters_after_quote}-after"
output_file = open(f'VS-and-Victorian-studies/quotation-contexts-{name_of_quote}.txt', mode='w', encoding='utf-8')

for context in context_chunks_for_quotations:
     output_file.write(context)
     output_file.write('\n')
output_file.close()

In [ ]:
#name_of_quote = f"{name_of_passage}-fulltexts"

## To write both contexts and IDs to a CSV


In [ ]:
zipped = list(zip(context_chunks_ids, context_chunks_decade, context_chunks_for_quotations))
chunks_with_ids_df = pd.DataFrame(zipped, columns=['id', 'Decade', 'Quotation Context Chunk'])
chunks_with_ids_df.to_csv(f'quotation-contexts-ids-{name_of_quote}.csv', encoding ='utf-8')

## To write output to multiple text files in a new directory

In [ ]:
# To output our context for quotations as a series of new files with the same beginning, followed by the number of the section

#Import pathlib 
from pathlib import Path

# Define and name the new output directory using pathlib
path = Path(f'{name_of_quote}_contexts/')
path.mkdir(exist_ok=True)

# Set the prefix for our output files, followed by the number of the section
begining_of_output_filenames = f'{name_of_quote}_contexts-'

# Iterate over each of the chunks of context for BookNLP NER
for i in range(1, len(context_chunks_for_quotations)+1):
    open(str(path) + "/" + begining_of_output_filenames+str(i)+'.txt','w').write(context_chunks_for_quotations[i-1])

## To look at most frequent words

In [ ]:
import re
from collections import Counter

# Define Functions

def split_into_words(any_chunk_of_text):
    lowercase_text = any_chunk_of_text.lower()
    split_words = re.split("\W+", lowercase_text) 
    return split_words

# Define Filepaths and Assign Variables

#directory_path = f'{name_of_quote}_contexts/'
filename = f'VS-and-Victorian-studies/quotation-contexts-{name_of_quote}.txt'
number_of_desired_words = 100

stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 've', 'll', 'amp',
]

most_frequent_non_stopwords = []


full_text = open(filename, encoding="utf-8", errors='ignore').read()
all_the_words = split_into_words(full_text)
non_stopwords = [word for word in all_the_words if word not in stopwords]
non_stopwords_tally = Counter(non_stopwords)
most_frequent_non_stopwords_to_add = non_stopwords_tally.most_common(number_of_desired_words)
most_frequent_non_stopwords.append(most_frequent_non_stopwords_to_add)

# Output Results

most_frequent_non_stopwords


In [ ]:
output_file = open(f'VS-and-Victorian-studies/quotation-contexts-most-frequent-words-{name_of_quote}.txt', mode='w', encoding='utf-8')

for item in most_frequent_non_stopwords:
    for word in item:
        print(word[0])
        output_file.write(word[0])
        output_file.write('\n')
output_file.close()

## To Topic Model our  contents

In [ ]:
#!pip install little_mallet_wrapper

In [ ]:
import little_mallet_wrapper
import seaborn
import glob
from pathlib import Path

## NOTE: we have to install openjdk + mallet
To install open Java Development Kit, either download from the 

## Step 1: Download and Install Java Development Kit

Go to the Java Development Kit download page, find your operating system, and click on the corresponding download link: https://www.oracle.com/java/technologies/javase-jdk14-downloads.html

Choose the most recent JDK (currently JDK 16) and click "JDK Download." This will take you to an installation page. Choose the appropriate file to download: 

- For Mac: macOS Installer (ARM is for Macs with M1/M2 chips, x64 is for Macs with Intel processors)
- For Windows:  Windowsx64 Installer
- For Linux: Windowsx64 Installer

Then open or unzip the file and follow all the instructions to complete the installation.

## Step 2: Tell your computer where to find Java

Now we have to tell our computers where to find JDK. For Macs, this involves defining a new variable, called JAVA_HOME and give it the filepath of our new JDK. For Windows, we edit a PATH variable to include the file path of JDK.

### For Mac

Open up your Terminal. Check whether the language at the top is "bash" or "zsh"

![image](../_images/command-line-language.png)

 - If the top of your command prompt shell says "bash"....
	-  You can run the following on the command line to set up the JAVA_HOME environment variable. The line of code adds your JAVA_HOME variable to a file called “bash_profile”, which is where environment variables are stored. `!echo "export JAVA_HOME=$(/usr/libexec/java_home)" >> ~/.bash_profile`
	-  Then, to immediately update your "bash profile", run: `source ~/.bash_profile`
	-  Then, to test whether Java correctly installed, type `javac`. If you get a list of options, you correctly installed it. If your command line says "Not recognized," then it is not correctly installed.
-  If the top of your command prompt shell says "zsh"...
	-  You can run the following on the command line to set up the JAVA_HOME environment variable. The line of code adds your JAVA_HOME variable to a file called “zshrc”, which is where environment variables are stored. `echo "export JAVA_HOME=$(/usr/libexec/java_home)" >> ~/.zshrc`
	-  Then, to immediately update your "zshrc", run: `source ~/.zshrc`
	-  Then, to test whether Java correctly installed, type `javac`. If you get a list of options, you correctly installed it. If your command line says "Not recognized," then it is not correctly installed.



##  To  install Mallet
To download MALLET, click the following [link](http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip) or find the link on the [MALLET home page.](https://mimno.github.io/Mallet/) Once the zip file downloads, unzip it.

Once installed, verify that it's working

To test whether MALLET works on your computer, type in the file path for MALLET on the command line and import-file, like so:

- Mac:  `~/mallet-2.0.8/bin/mallet import-file` 
- Windows: `C:\mallet-2.0.8.\bin\mallet import-file`

If it’s working, then you’ll get a message that says “A tool for creating instance lists of feature vectors from comma-separated-values” and a list of options.

If you get a permissions error, type `chmod +x mallet-2.0.8/bin/mallet`

In [ ]:
path_to_mallet = 'mallet-2.0.8/bin/mallet'

In [ ]:
directory = f'{name_of_quote}_contexts/'

In [ ]:
files = glob.glob(f"{directory}/*.txt")

In [ ]:
files

In [ ]:
training_data = []
for file in files:
    text = open(file, encoding='utf-8').read()
    processed_text = little_mallet_wrapper.process_string(text, numbers='remove')
    training_data.append(processed_text)

In [ ]:
original_texts = []
for file in files:
    text = open(file, encoding='utf-8').read()
    original_texts.append(text)

In [ ]:
context_titles = [Path(file).stem for file in files]
# The Path().stem function extract the filename without the .txt extension

In [ ]:
context_titles

In [ ]:
little_mallet_wrapper.print_dataset_stats(training_data)

In [ ]:
num_topics = 5 # Change this number to change the number of topicsnum_topics = 10 # Change this number to change the number of topics

In [ ]:
training_data = training_data

In [ ]:
#Change to your desired output directory
output_directory_path = f'topic-model-output/{name_of_quote}'

#No need to change anything below here
Path(f"{output_directory_path}").mkdir(parents=True, exist_ok=True)

path_to_training_data           = output_directory_path + '/training.txt'
path_to_formatted_training_data = output_directory_path + '/mallet.training'
path_to_model                   = output_directory_path + '/mallet.model.' + str(num_topics)
path_to_topic_keys              = output_directory_path + '/mallet.topic_keys.' + str(num_topics)
path_to_topic_distributions     = output_directory_path + '/mallet.topic_distributions.' + str(num_topics)
path_to_word_weights            = output_directory_path + '/mallet.word_weights.' + str(num_topics)
path_to_diagnostics             = output_directory_path + '/mallet.diagnostics.' + str(num_topics) + '.xml'



In [ ]:
little_mallet_wrapper.import_data(path_to_mallet,
                path_to_training_data,
                path_to_formatted_training_data,
                training_data)

In [ ]:
little_mallet_wrapper.train_topic_model(path_to_mallet,
                      path_to_formatted_training_data,
                      path_to_model,
                      path_to_topic_keys,
                      path_to_topic_distributions,
                      path_to_word_weights,
                      path_to_diagnostics,
                      num_topics)

### Display Topics and Top Words

In [ ]:
topics = little_mallet_wrapper.load_topic_keys(path_to_topic_keys)

for topic_number, topic in enumerate(topics):
    print(f"Topic {topic_number}\n\n{topic}\n")

## Load Topic Distributions

In [ ]:
topic_distributions = little_mallet_wrapper.load_topic_distributions(path_to_topic_distributions)

In [ ]:
#Let's look at number 5
topic_distributions[5]

In [ ]:
context_to_check = f"{name_of_quote}_contexts-6"

chunk_number = context_titles.index(context_to_check)

print(f"Topic Distributions for {context_titles[chunk_number]}\n")

print("Chunk contents:\n")
print(open(f'{directory}/{context_to_check}.txt', 'r').read())
print('\n')
for topic_number, (topic, topic_distribution) in enumerate(zip(topics, topic_distributions[chunk_number])):
    print(f"Topic {topic_number} {topic[:20]} \nProbability: {round(topic_distribution, 3)}\n")

### Top context chunks

In [ ]:
training_data_context_titles = dict(zip(training_data, context_titles))
training_data_original_text = dict(zip(training_data, original_texts))

In [ ]:
def display_top_titles_per_topic(topic_number=0, number_of_documents=5):
    
    print(f"Topic {topic_number}\n\n{topics[topic_number]}\n")

    for probability, document in little_mallet_wrapper.get_top_docs(training_data, topic_distributions, topic_number, n=number_of_documents):
        print(round(probability, 4), training_data_context_titles[document] + "\n")
    return

In [ ]:
display_top_titles_per_topic(topic_number=0, number_of_documents=7)

In [ ]:
display_top_titles_per_topic(topic_number=1, number_of_documents=5)

In [ ]:
display_top_titles_per_topic(topic_number=2, number_of_documents=6)

In [ ]:
display_top_titles_per_topic(topic_number=3, number_of_documents=6)

In [ ]:
display_top_titles_per_topic(topic_number=4, number_of_documents=6)

In [ ]:
display_top_titles_per_topic(topic_number=5, number_of_documents=6)

In [ ]:
display_top_titles_per_topic(topic_number=6, number_of_documents=6)

In [ ]:
display_top_titles_per_topic(topic_number=7, number_of_documents=6)

In [ ]:
display_top_titles_per_topic(topic_number=8, number_of_documents=6)

In [ ]:
display_top_titles_per_topic(topic_number=9, number_of_documents=6)